In [4]:
import torch
import numpy as np

In [3]:
# check status
 
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce GTX 1060 3GB


In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Sequential, Linear, ReLU, MSELoss, L1Loss, CrossEntropyLoss, NLLLoss, LogSoftmax, Softmax
from torch.optim import Adam, RMSprop, SGD
from torch import from_numpy, no_grad, argmax
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from sklearn import preprocessing

In [7]:
class NNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        ##############
        ##############

        self.fc1 = nn.Linear(25, 6)
        self.fcout = nn.Linear(6, 3)

        ##############
        ##############

    def forward(self, x):
        ##############
        ##############

        x = F.relu(self.fc1(x))
        x = F.softmax(self.fcout(x), dim=1)

        ##############
        ##############
        return x


def main_cpu():
    torch.manual_seed(123)
    BATCH_SIZE = 30
    EPOCHS = 50
    LEARNING_RATE = 0.001

    data = np.load("biofilm_data.npy")
    X = data[:, :-1]
    y = data[:, -1]
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)

    X_train, X_val = torch.from_numpy(X[:int(X.shape[0] * 0.8), :].astype(np.float)).float(), torch.from_numpy(
        X[int(X.shape[0] * 0.8):, :].astype(np.float)).float()
    y_train, y_val = torch.from_numpy(y[:int(X.shape[0] * 0.8)].astype(np.float)).float(), torch.from_numpy(
        y[int(X.shape[0] * 0.8):].astype(np.float)).float()

    dl_training = None
    dl_validation = None

    ##############
    ##############

    dl_training = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
    dl_validation = DataLoader(TensorDataset(X_val, y_val), batch_size=BATCH_SIZE, shuffle=True)

    ##############
    ##############

    standard_model = NNClassifier()

    optimizer = Adam(standard_model.parameters(), lr=LEARNING_RATE)  # can be replaced with RMSprop, SGD..
    loss_fun = CrossEntropyLoss()

    for epoch in range(EPOCHS):
        epoch_loss = 0.0
        for inputs, true_output in dl_training:
            predictions = None
            loss = None
            ##############
            ##############

            predictions = standard_model(inputs)
            loss = loss_fun(predictions, true_output.type(torch.LongTensor))

            ##############
            ##############
            epoch_loss += float(loss.detach())

            ##############
            ##############

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            ##############
            ##############
        with no_grad():
            standard_model.eval()
            acc = 0.0
            for inputs, true_output in dl_training:
                predictions = standard_model(inputs)

                acc += (argmax(predictions, -1) == true_output).sum()

        with no_grad():
            standard_model.eval()
            val_acc = 0.0
            for inputs, true_output in dl_validation:
                predictions = standard_model(inputs)
                val_acc += (argmax(predictions, -1) == true_output).sum()
        standard_model.train()

        print(["Epoch: %d, Training accuracy: %3.4f, Validation accuracy: %3.4f" % (
            epoch + 1, acc * 100 / len(dl_training.dataset), val_acc * 100 / len(dl_validation.dataset))])

    torch.save(standard_model.state_dict(), "my_classification_model.pt")  # DO NOT MODIFY THIS LINE!
    ##############
    ##############

    # TODO: add saved model to repository! (no coding necessary, just wanted to get your attention here)

    ##############
    ##############

In [8]:
import timeit

start = timeit.default_timer()

#Your statements here
main_cpu()

stop = timeit.default_timer()

print('Time: ', stop - start) 

['Epoch: 1, Training accuracy: 53.9527, Validation accuracy: 48.8599']
['Epoch: 2, Training accuracy: 68.1337, Validation accuracy: 58.3062']
['Epoch: 3, Training accuracy: 68.3782, Validation accuracy: 58.3062']
['Epoch: 4, Training accuracy: 68.3782, Validation accuracy: 58.3062']
['Epoch: 5, Training accuracy: 68.3782, Validation accuracy: 57.9805']
['Epoch: 6, Training accuracy: 68.3782, Validation accuracy: 57.9805']
['Epoch: 7, Training accuracy: 68.3782, Validation accuracy: 58.3062']
['Epoch: 8, Training accuracy: 68.2967, Validation accuracy: 58.3062']
['Epoch: 9, Training accuracy: 68.2967, Validation accuracy: 58.3062']
['Epoch: 10, Training accuracy: 68.2967, Validation accuracy: 58.3062']
['Epoch: 11, Training accuracy: 68.2967, Validation accuracy: 58.3062']
['Epoch: 12, Training accuracy: 68.2967, Validation accuracy: 58.3062']
['Epoch: 13, Training accuracy: 68.2967, Validation accuracy: 58.3062']
['Epoch: 14, Training accuracy: 68.3782, Validation accuracy: 58.3062']
[